# Random Forest

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("dataset/CL-dataset.csv",sep='\t',decimal=",",index_col=0)

In [15]:
df.head()

,Imax,E,PIL,MeanD,EDate,class
0,48,5.121928,2.567,111.000000,0.848548,0
1,25,6.125617,2.567,60.333333,1.468334,0
2,24,4.310443,2.567,15.500000,0.000000,0
3,72,4.700440,2.567,77.000000,0.995727,0
4,48,5.831955,2.567,37.200000,1.494785,0


In [16]:
from sklearn.model_selection import train_test_split
label = df.pop('class')
train_set, test_set, train_label, test_label = train_test_split(df, label, stratify =label, test_size=0.30)

In [17]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


## Grid search

In [18]:


param_dist = [{"max_depth": [2,3,12,None],
              "max_features": [5],# Al massimo 6 non di più
              "min_samples_split": [10,50],
              "min_samples_leaf": [10,50],
              "bootstrap": [True, False],
              "criterion": ["entropy", "gini"],
              "class_weight":['balanced', None, {0: 0.3, 1: 0.7}]}]

clf = RandomForestClassifier(n_estimators=30)
grid_search = GridSearchCV(clf, param_dist,  
                            n_jobs=3, 
                            scoring='accuracy')

grid_search.fit(train_set, train_label)

print("Complete Grid search:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

print("Best parameters:",grid_search.best_params_)
print("Best Score: ",grid_search.best_score_)


samples_leaf': 10, 'min_samples_split': 10}
0.488 (+/-0.049) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 2, 'max_features': 5, 'min_samples_leaf': 10, 'min_samples_split': 50}
0.490 (+/-0.054) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 2, 'max_features': 5, 'min_samples_leaf': 50, 'min_samples_split': 10}
0.485 (+/-0.032) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 2, 'max_features': 5, 'min_samples_leaf': 50, 'min_samples_split': 50}
0.507 (+/-0.045) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 3, 'max_features': 5, 'min_samples_leaf': 10, 'min_samples_split': 10}
0.494 (+/-0.041) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 3, 'max_features': 5, 'min_samples_leaf': 10, 'min_samples_split': 50}
0.497 (+/-0.020) for {'bootstrap': True, 'class_weight':

## Modello Finale

In [19]:
rf = RandomForestClassifier(n_estimators=30, 
                             criterion='entropy',
                             class_weight=None,
                             max_features=5,
                             max_depth=12, 
                             min_samples_split=50,
                             min_samples_leaf=10,
                             bootstrap=True) 
rf = rf.fit(train_set, train_label)

In [20]:
#prediction on the test test
test_pred_rf = rf.predict(test_set)

In [21]:
#compute the performance of the model
print(classification_report(test_label, 
                            test_pred_rf))

              precision    recall  f1-score   support

           0       0.67      0.61      0.64       507
           1       0.52      0.48      0.50       507
           2       0.63      0.75      0.68       507

    accuracy                           0.61      1521
   macro avg       0.61      0.61      0.61      1521
weighted avg       0.61      0.61      0.61      1521

